In [12]:
import chromadb
from chromadb.utils import embedding_functions
from langchain_ollama import OllamaEmbeddings,OllamaLLM
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
chroma_client = chromadb.PersistentClient(path="chroma_google_genai")

In [3]:
embeddings = OllamaEmbeddings(model="embeddinggemma:300m")

In [4]:
embedding_func = embedding_functions.OllamaEmbeddingFunction(model_name="embeddinggemma:300m")
collection = chroma_client.get_or_create_collection(name="chroma_google_genai",embedding_function=embedding_func)

In [13]:
llm = OllamaLLM(model="qwen3:8b",temperature=0)
llm2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",temperature=0)

In [6]:
from langchain_community.vectorstores import Chroma

ret = Chroma(client=chroma_client,collection_name="chroma_google_genai",embedding_function=embeddings)

C:\Users\hp\AppData\Local\Temp\ipykernel_16412\725621917.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  ret = Chroma(client=chroma_client,collection_name="chroma_google_genai",embedding_function=embeddings)


In [7]:
from langchain_classic.chains import RetrievalQA


In [16]:
qa_chain = RetrievalQA.from_chain_type(
   retriever=ret.as_retriever(),
   llm=llm2,
   chain_type="stuff"
)

In [17]:
question = input("Enter your question")
try:
    print(qa_chain.invoke(question))
except Exception as e:
    print(f"Error: {e}")

{'query': 'schemes in rajastahan', 'result': "Rajasthan has a wide range of schemes implemented by both the state and central governments, covering various sectors like agriculture, health, education, social welfare, employment, and infrastructure. Here's a breakdown of some prominent schemes, categorized for clarity:\n\n**I. Agriculture and Rural Development:**\n\n*   **Kisan Samman Nidhi (PM-KISAN):** A central government scheme providing direct income support to all landholding farmer families.\n*   **Mukhyamantri Krishi Parivar Sinchai Yojana:** Aims to promote micro-irrigation techniques like drip and sprinkler systems to conserve water and increase crop productivity.\n*   **Rajiv Gandhi Kisan Beej Guarantee Yojana:** Focuses on providing quality seeds to farmers.\n*   **Mukhyamantri Jal Swavlamban Abhiyan (MJSA):** A flagship scheme for water conservation and harvesting in rural areas.\n*   **National Food Security Mission (NFSM):** Aims to increase the production of rice, wheat,

In [27]:
from langchain_classic.chains import create_retrieval_chain     # ← Classic package
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain  # ← Classic package
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on the provided documents. "
               "If you can't find the answer in the context, say so."),
    MessagesPlaceholder(variable_name="context"),  # Retrieved docs go here
    ("human", "{input}")  # ← This provides the required HumanMessage
])
prompt = ChatPromptTemplate.from_messages([("system", prompt)])
question_answer_chain = create_stuff_documents_chain(llm2, prompt)
qa_chain = create_retrieval_chain(ret.as_retriever(), question_answer_chain)

ValueError: Invalid template: input_variables=['context', 'input'] input_types={'context': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [28]:
response = qa_chain.invoke({"input": "What is prompt engineering?"})
print(response["answer"])

ValueError: No content messages found. The Gemini API requires at least one non-system message (HumanMessage, AIMessage, etc.) in addition to any SystemMessage. Please include additional messages in your input.